I have developed this code by studying a few other Kernels based on CatBoost and XGBoost models. This approach is based on preparing the training set where for each assessment session, the corresponding row contains details of user activities prior to that assessment, for example activities completed, time spent etc. The model is very simplistic and it is running fine on my laptop. But when I submit it, **I get error "submission.csv" not found** which is a bit disappointing. I have done a number of changes to reduce memory usage. The code runs fine on my laptop and also the commit process finishes in decent time. But that has not been of help. Moreover, the original Kernel from which I created this one was succesfully submitted from my machine but with my changes it fails. So, I am not able to get a public score. 

My plan was to progressively add new features and improve the model, but without a public score I can't assess where I stand. I am quite new to Kaggle, this is my second submission. But have checked some of the basic things like format of submission.csv, also the code does not contain any specific reference to the size of the test file (which I believe could create a problem because the private test file is much larger). Can you please help me figure what I am doing wrong here? Thanks a lot! :)

In [ ]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import cohen_kappa_score
from scipy.stats import mode
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split


import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot
import shap


### <font size=4 color='violet'> Reading and understanding our data</font>

In [ ]:
%%time
keep_cols = ['game_session', 'installation_id','event_code', 'title', 'game_time', 'type']
train=pd.read_csv('../input/data-science-bowl-2019/train.csv',usecols=keep_cols)
train_labels=pd.read_csv('../input/data-science-bowl-2019/train_labels.csv',
                         usecols=['installation_id','game_session','accuracy_group'])
test=pd.read_csv('../input/data-science-bowl-2019/test.csv',usecols=keep_cols)
submission=pd.read_csv('../input/data-science-bowl-2019/sample_submission.csv')

In [ ]:
not_req=(set(train.installation_id.unique()) - set(train_labels.installation_id.unique()))

In [ ]:
train_new=~train['installation_id'].isin(not_req)
train.where(train_new,inplace=True)
train.dropna(inplace=True)
train['event_code']=train.event_code.astype(int)

In [ ]:
# test_id =('8be4aedf',  '5d21020f',  'bfddcc77',  '75857694', 'b4738558', '2db60743', 'a5ac9b55', '3d214090', '50c78eb8', '48166507', '3713a3ae')
# test_new=test[test['installation_id'].isin(test_id)]
# test=test_new

In [ ]:
activityTypes= train[['title','type']].drop_duplicates()
#list_of_activities = list(set(train['title'].value_counts().index))

In [ ]:
def getFeatures(userData, isTest):
    counter = 0
    #test_set=False
    session_List=[]

    for i, (ins_id, user_sample) in enumerate(userData.groupby('installation_id', sort=False)):
        session_type_counter = dict({'Clip':0, 'Activity': 0, 'Game': 0, 'Assessment': 0})
        time_spent = 0
        clipTime=0
        activityTime=0
        gameTime=0
        assessmentTime=0

        activityLog = dict()
        for i, row in activityTypes.iterrows():
            activityLog[row['title']]=0
            activityLog[row["title"]+"Time"]=0
           # if ("Assessment" in row['title']):
            #    activityLog[row["title"]+"Solved"]=0


        for i, session in user_sample.groupby('game_session', sort=False):
            session_type = session['type'].iloc[0]
            session_title = session['title'].iloc[0]
            session_id = session['game_session'].iloc[0]
            processAssessment=False
            if (isTest) & (len(session)==1) & (session_type == 'Assessment'):
                processAssessment = True
                #print('checkpoint1 ', session_type, '  ', len(session), ' ', session_id)
            elif (~isTest) & (len(session) > 1):
                processAssessment = True
            else:
                processAssessment= False
            

            #if (session_type == 'Assessment'):
                #all_attempts = session.query(f'event_code == {activity_correct_event_code[session_title]}')
                #true_attempts = all_attempts['event_data'].str.contains('true').sum()
                #false_attempts = all_attempts['event_data'].str.contains('false').sum()

            if (session_type == 'Assessment') & (processAssessment):
                Dict1 =dict({'installation_id': session['installation_id'].iloc[0]})
                Dict1['game_session']=session['game_session'].iloc[0]

                # Dict1['type']=session['type'].iloc[0]
                Dict1['title']=session['title'].iloc[0]
                Dict1['priorClips']=session_type_counter['Clip']
                Dict1['priorActivity']=session_type_counter['Activity']
                Dict1['priorGames']=session_type_counter['Game']
                Dict1['priorAssessments']=session_type_counter['Assessment']
                #Dict1['session_time']= session['game_time'].max()
                #Dict1['total_prior_time']=time_spent
    #            Dict1['avgclipTime']=clipTime/session_type_counter['Clip']
    #            Dict1['avgactivityTime']=activityTime/session_type_counter['Activity']
    #            Dict1['avggameTime']= gameTime/session_type_counter['Game']
    #            Dict1['avgassessmentTime']=assessmentTime/session_type_counter['Assessment']
                Dict1.update(activityLog)
                if (isTest):
                    session_List.append(Dict1)
                else:
                    #if all_attempts > 0:
                    session_List.append(Dict1)
                              
            if len(session)>1:
                    session_type_counter[session_type]+=1
                    time_spent+=session['game_time'].max()
                    activityLog[session_title]+=1
                    activityLog[session_title+"Time"]+=session['game_time'].max()

                    if(session_type=="Clip"):
                        clipTime+=session['game_time'].max()
                    elif(session_type=="Game"):
                        gameTime+=session['game_time'].max()
                    elif(session_type=="Activity"):
                        activityTime+=session['game_time'].max()
                    elif(session_type=="Assessment"):
                        assessmentTime+=session['game_time'].max()
                       # if true_attempts > 0:
                        #    activityLog[session_title+"Solved"]+=1
    processedDf=pd.DataFrame(session_List)
    return processedDf
# featuresDf.head(20)
# print(len(session_List))
# featuresDf=pd.DataFrame(session_List)
# featuresDf.head(20)

In [ ]:
final_train = getFeatures(train, False)

In [ ]:
del train

In [ ]:
final_test = getFeatures(test, True)

In [ ]:
del test

In [ ]:
final_train=pd.merge(train_labels, final_train, on=['game_session', 'installation_id'], how='left')

In [ ]:
id_columns=['game_session', 'installation_id']
drop_columns=['num_correct', 'num_incorrect', 'accuracy']
cat_columns=['title']

In [ ]:
final_train.drop(id_columns, axis=1, inplace=True)
#final_train.drop(drop_columns, axis=1, inplace=True)

In [ ]:
final_test.drop(id_columns, axis=1, inplace=True)

In [ ]:
final=pd.concat([final_train,final_test])
encoding=['title']
for col in encoding:
    lb=LabelEncoder()
    lb.fit(final[col])
    final[col]=lb.transform(final[col])
    
final_train=final[:len(final_train)]
final_test=final[len(final_train):]


In [ ]:
X_train=final_train.drop('accuracy_group',axis=1)
y_train=final_train['accuracy_group']

In [ ]:
def model(X_train,y_train,final_test,n_splits=3):
    scores=[]
    pars = {
        'colsample_bytree': 0.8,                 
        'learning_rate': 0.08,
        'max_depth': 10,
        'subsample': 1,
        'objective':'multi:softprob',
        'num_class':4,
        'eval_metric':'mlogloss',
        'min_child_weight':3,
        'gamma':0.25,
        'n_estimators':500
    }

    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    y_pre=np.zeros((len(final_test),4),dtype=float)
    final_test=xgb.DMatrix(final_test.drop('accuracy_group',axis=1))


    for train_index, val_index in kf.split(X_train, y_train):
        train_X = X_train.iloc[train_index]
        val_X = X_train.iloc[val_index]
        train_y = y_train[train_index]
        val_y = y_train[val_index]
        xgb_train = xgb.DMatrix(train_X, train_y)
        xgb_eval = xgb.DMatrix(val_X, val_y)

        xgb_model = xgb.train(pars,
                      xgb_train,
                      num_boost_round=1000,
                      evals=[(xgb_train, 'train'), (xgb_eval, 'val')],
                      verbose_eval=False,
                      early_stopping_rounds=20
                     )

        val_X=xgb.DMatrix(val_X)
        pred_val=[np.argmax(x) for x in xgb_model.predict(val_X)]
        score=cohen_kappa_score(pred_val,val_y,weights='quadratic')
        scores.append(score)
        print('choen_kappa_score :',score)

        pred=xgb_model.predict(final_test)
        y_pre+=pred

    pred = np.asarray([np.argmax(line) for line in y_pre])
    print('Mean score:',np.mean(scores))
    
    return xgb_model,pred

In [ ]:
xgb_model,pred=model(X_train,y_train,final_test,5)

In [ ]:
sub=pd.DataFrame({'installation_id':submission.installation_id,'accuracy_group':pred})
sub.to_csv('submission.csv',index=False)


## <font size=5 color='violet'>Feature Selection</font>

We will use    module of xgboost to plot the feature importances and see what features  our model think are important for making prediction.


In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
xgb.plot_importance(xgb_model, max_num_features=50, height=0.5, ax=ax,importance_type='gain')
plt.show()

There are three methods to measure feature_importances in xgboost.They are :
- `weight` : The total number of times this feature was used to split the data across all trees.
- `Cover` :The number of times a feature is used to split the data across all trees weighted by the number of training data points that go through those splits.
- `Gain` : The average loss reduction gained when using this feature for splitting in trees.

We used `Gain` in the above example and the model says when it used `event_code_2030` the loss on average was reduced by 8%.

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
xgb.plot_importance(xgb_model, max_num_features=50, height=0.5, ax=ax,importance_type='weight')
plt.show()

When we considered weight,the model says that is used `game_time mean` 1035 times to split the data across the trees.

hmmm...so how what can we conclude from above figures?
We will find out....

In [ ]:
# shap_values = shap.TreeExplainer(xgb_model).shap_values(X_train)
# shap.summary_plot(shap_values, X_train, plot_type="bar")

In [ ]:
# shap.summary_plot(shap_values[3], X_train)

In [ ]:
# shap.summary_plot(shap_values[0], X_train)

In [ ]:
# X_train,X_test,y_train,y_test=train_test_split(X_train,y_train,test_size=.1)
# model = XGBClassifier()
# model.fit(X_train, y_train)

In [ ]:
# threshold = np.sort(model.feature_importances_)[40:]
# for thresh in threshold:
#     # select features using threshold
#     selection = SelectFromModel(model, threshold=thresh, prefit=True)
#     select_X_train = selection.transform(X_train)
#     # train model
#     selection_model = XGBClassifier()
#     selection_model.fit(select_X_train, y_train)
#     # eval model
#     select_X_test = selection.transform(X_test)
#     y_pred = selection_model.predict(select_X_test)
#     predictions = [round(value) for value in y_pred]
#     accuracy = cohen_kappa_score(y_test, predictions)
#     print("Thresh=%.3f, n=%d, cohen kappa score: %.2f%%" % (thresh, select_X_train.shape[1], accuracy*100.0))
    
    
    